In [55]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
import gensim
from gensim.models import CoherenceModel
from gensim import corpora
import pandas as pd
from pprint import pprint
import string
import os
import re
import  sys
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis


In [56]:
# Add parent directory to path to import modules from src
rpath = os.path.abspath('..')
if rpath not in sys.path:
    sys.path.insert(0, rpath)

from src.loader import SlackDataLoader
import src.utils as utils

In [57]:
data_loader = SlackDataLoader("../data")

def get_channel_messages(channel):
    channel_messages = utils.get_messages_on_channel(f"../data/{channel}") 
    # Create an empty DataFrame
    df = pd.DataFrame(channel_messages)
    return df

def get_all_channels_message():
    dfs = []  # List to store individual DataFrames

    for channel in data_loader.channels:
        dfs.append(get_channel_messages(channel["name"]))

    # Concatenate all DataFrames into a single DataFrame
    result_df = pd.concat(dfs, ignore_index=True)

    return result_df

In [59]:
def preprocess_text(text):
    # Extract and remove URLs
    urls = re.findall(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text)
    for url in urls:
        text = text.replace(url, '')

    text = re.sub(r'<@.*?>', '', text)

    # Convert to lowercase
    text = text.lower()

    # Remove punctuation
    text = ''.join([char for char in text if char not in string.punctuation])

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Tokenize
    tokens = word_tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Perform stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]

    # Perform lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Join the tokens back into a string
    text = ' '.join(tokens)

    return text


In [61]:

def prepare_data(df):
    df['cleaned_text'] = df['text'].apply(preprocess_text)
    sentence_list = [tweet for tweet in df['cleaned_text']]
    word_list = [sent.split() for sent in sentence_list]

    #Create dictionary which contains Id and word
    word_to_id = corpora.Dictionary(word_list) #generate unique tokens
    corpus = [word_to_id.doc2bow(tweet) for tweet in word_list]
    
    return df, word_list, word_to_id, corpus

In [63]:
def build_model(corpus, word_to_id):
    # Build LDA model
    lda_model = gensim.models.ldamodel.LdaModel(corpus,
                                        id2word=word_to_id,
                                        num_topics=5,
                                        random_state=100,
                                        update_every=1,
                                        chunksize=100,
                                        passes=10,
                                        alpha='auto',
                                        per_word_topics=True)    
    return lda_model

In [65]:

def show_topics(lda_model):
    pprint(lda_model.show_topics(formatted=False))

In [66]:
def model_analysis(lda_model, corpus, word_list, word_to_id):
    print('\nPerplexity: ', lda_model.log_perplexity(corpus))
    doc_lda = lda_model[corpus]


    # Compute Coherence Score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=word_list, dictionary=word_to_id, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print('\n Lda model Coherence Score/Accuracy on Tweets: ', coherence_lda)

In [84]:
def get_top_topics(df):
    df, word_list, word_to_id, corpus = prepare_data(df)
    lda_model = build_model(corpus, word_to_id)

    # Show the top 10 topics
    show_topics(lda_model)
    

    # Visualize the top 10 topics
    pyLDAvis.enable_notebook()
    LDAvis_prepared = gensimvis.prepare(lda_model, corpus, word_to_id)
    return LDAvis_prepared

### Top 10 topics of the different channels

In [85]:
df = get_channel_messages("all-week1")
get_top_topics(df)

[(0,
  [('use', 0.04629722),
   ('user', 0.034003064),
   ('session', 0.028530736),
   ('featur', 0.018247005),
   ('score', 0.015890928),
   ('total', 0.014961363),
   ('•', 0.014906585),
   ('start', 0.014091308),
   ('engag', 0.013270916),
   ('think', 0.013084882)]),
 (1,
  [('work', 0.04845276),
   ('channel', 0.032532107),
   ('two', 0.018797847),
   ('file', 0.018182827),
   ('use', 0.01643002),
   ('data', 0.015751915),
   ('notebook', 0.012717859),
   ('incognito', 0.01247491),
   ('add', 0.011952376),
   ('expect', 0.011469893)]),
 (2,
  [('thank', 0.06351325),
   ('column', 0.024919178),
   ('id', 0.023483997),
   ('valu', 0.022867942),
   ('data', 0.019010132),
   ('use', 0.016901003),
   ('link', 0.016111318),
   ('submiss', 0.015757607),
   ('github', 0.014633137),
   ('task', 0.0133222)]),
 (3,
  [('normal', 0.02755062),
   ('use', 0.02691801),
   ('outlier', 0.02512235),
   ('report', 0.025107931),
   ('tri', 0.021214876),
   ('mean', 0.017638601),
   ('want', 0.0169880

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.000712 -0.253408       1        1  27.301395
0     -0.107441  0.069621       2        1  22.082541
4      0.257197  0.071508       3        1  21.231041
1     -0.103790  0.096131       4        1  15.108001
3     -0.046677  0.016149       5        1  14.277022, topic_info=        Term       Freq      Total Category  logprob  loglift
131    thank  69.000000  69.000000  Default  30.0000  30.0000
168     work  29.000000  29.000000  Default  29.0000  29.0000
430     user  30.000000  30.000000  Default  28.0000  28.0000
45   channel  20.000000  20.000000  Default  27.0000  27.0000
36   session  25.000000  25.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
413    group   4.468646   7.515580   Topic5  -4.8404   1.4266
92     think   8.361287  33.839462   Topic5  -4.2139   0.5485
19     could   5.456864  12.473370   Topic5  -4.6406   1.1198
44        ye   5.064465  11.095911   Topic5  -4.7152   1.1622
20      data   7.040192  47.849470   Topic5  -4.3858   0.0301

[235 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
693       3  0.961259  account
392       4  0.892204      add
783       5  0.906388   affect
436       5  0.838046      agg
73        2  0.800737   aggreg
...     ...       ...      ...
85        4  0.930302     yeah
441       5  0.636535        ‘
431       5  0.891415        ’
545       4  0.883441        ”
432       2  0.948450        •

[242 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 5, 2, 4])

In [86]:
df = get_channel_messages("all-community-building")
get_top_topics(df)

[(0,
  [('arun', 0.042168994),
   ('get', 0.021699265),
   ('dont', 0.020860696),
   ('first', 0.01833718),
   ('know', 0.016804324),
   ('think', 0.0153800035),
   ('joy', 0.0148091195),
   ('say', 0.014181729),
   ('go', 0.013683029),
   ('man', 0.012136477)]),
 (1,
  [('u', 0.04294473),
   ('one', 0.035910558),
   ('like', 0.03278028),
   ('thank', 0.02106849),
   ('make', 0.020496279),
   ('grin', 0.018016053),
   ('day', 0.016980072),
   ('person', 0.014324365),
   ('realli', 0.01369259),
   ('would', 0.013683403)]),
 (2,
  [('good', 0.090647586),
   ('morn', 0.07716033),
   ('time', 0.036429323),
   ('cb', 0.023522833),
   ('•', 0.022947542),
   ('today', 0.020783931),
   ('session', 0.018044068),
   ('plea', 0.017591298),
   ('happi', 0.014716192),
   ('let', 0.014710607)]),
 (3,
  [('hello', 0.10202492),
   ('week', 0.03399055),
   ('guy', 0.022563318),
   ('peopl', 0.019647341),
   ('use', 0.018128946),
   ('friend', 0.017528538),
   ('last', 0.016712157),
   ('laugh', 0.01351

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.109805  0.281216       1        1  25.860723
1     -0.092242 -0.192213       2        1  22.055512
2      0.324171  0.019241       3        1  19.223308
3     -0.072113 -0.067784       4        1  17.705111
4     -0.050010 -0.040460       5        1  15.155346, topic_info=        Term        Freq       Total Category  logprob  loglift
206    hello  638.000000  638.000000  Default  30.0000  30.0000
5       good  616.000000  616.000000  Default  29.0000  29.0000
10      morn  524.000000  524.000000  Default  28.0000  28.0000
2183    arun  386.000000  386.000000  Default  27.0000  27.0000
245        u  335.000000  335.000000  Default  26.0000  26.0000
...      ...         ...         ...      ...      ...      ...
34     googl   26.673319   27.361135   Topic5  -5.3019   1.8614
82    someth   25.931579   26.608907   Topic5  -5.3301   1.8610
1016    wink   25.870769   26.564880   Topic5  -5.3325   1.8603
3038   delet   40.204450   41.956864   Topic5  -4.8916   1.8442
1025  common   28.439291   30.571426   Topic5  -5.2378   1.8145

[186 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
599       2  0.996769  academi
332       1  0.988104     also
0         3  0.994910  announc
189       4  0.992792   answer
2183      1  0.996191     arun
...     ...       ...      ...
1233      5  0.978832     year
85        5  0.988657        ’
2911      4  0.991802        “
2912      4  0.984405        ”
33        3  0.996703        •

[161 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5])

### Top 10 topics of all channels

In [87]:
df = get_all_channels_message()
get_top_topics(df)

[(0,
  [('work', 0.061310686),
   ('thank', 0.035198063),
   ('think', 0.03282949),
   ('task', 0.020581743),
   ('im', 0.017542895),
   ('logo', 0.015831266),
   ('dont', 0.015652644),
   ('also', 0.015453295),
   ('code', 0.014646822),
   ('model', 0.013476593)]),
 (1,
  [('channel', 0.18876673),
   ('join', 0.18849722),
   ('use', 0.050648026),
   ('data', 0.025946053),
   ('ye', 0.022727432),
   ('connect', 0.018275846),
   ('plea', 0.017530445),
   ('time', 0.017410634),
   ('start', 0.016544862),
   ('sure', 0.0149129685)]),
 (2,
  [('let', 0.032837767),
   ('guy', 0.03207949),
   ('good', 0.030575136),
   ('u', 0.030185215),
   ('make', 0.026710935),
   ('group', 0.026556589),
   ('featur', 0.02613478),
   ('need', 0.025963098),
   ('today', 0.023980152),
   ('run', 0.023953129)]),
 (3,
  [('file', 0.04864594),
   ('tri', 0.037348438),
   ('line', 0.029828414),
   ('error', 0.02105171),
   ('instanc', 0.019424342),
   ('get', 0.01919499),
   ('imag', 0.013803347),
   ('creat', 0

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.371835  0.040003       1        1  23.667121
3     -0.147841  0.284445       2        1  23.585262
0     -0.088493 -0.263332       3        1  21.320806
4     -0.081064 -0.020587       4        1  16.903566
2     -0.054437 -0.040528       5        1  14.523244, topic_info=         Term         Freq        Total Category  logprob  loglift
523   channel  4514.000000  4514.000000  Default  30.0000  30.0000
56       join  4507.000000  4507.000000  Default  29.0000  29.0000
178      work  1321.000000  1321.000000  Default  28.0000  28.0000
166       use  1211.000000  1211.000000  Default  27.0000  27.0000
4447     file  1159.000000  1159.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
639      said   105.777951   106.561229   Topic5  -4.9324   1.9220
829      hour   103.377734   104.157329   Topic5  -4.9554   1.9219
227     clear   102.639071   103.413959   Topic5  -4.9625   1.9219
87      thing   100.690676   101.468603   Topic5  -4.9817   1.9217
2051        g   119.037826   159.760959   Topic5  -4.8143   1.6352

[189 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
1053      4  0.995779        abl
5357      2  0.995068     access
2397      4  0.995778         ad
2355      1  0.989266    airflow
332       3  0.998024       also
...     ...       ...        ...
55        5  0.996958       yeah
619       1  0.989362  yesterday
200       3  0.994788        yet
85        4  0.992700          ’
33        1  0.997926          •

[188 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 1, 5, 3])